In [30]:
###CODE WRITTEN BY ZACHARY SORENSON###

import numpy as np
from numpy import *
import scipy as sp
import pandas as pd
import matplotlib as mpl
import sys
import math
import re
from enum import Enum
from itertools import cycle, islice, dropwhile
import random
import time
import statistics
from itertools import permutations
import pyomo.environ as pe
from pyomo.environ import *
from pyomo.opt import SolverFactory
import tsplib95
import networkx

# NOTE: Compile the last block of code before running any subroutines

# Success Rate Subroutine

In [34]:
# # load the tsplib problem
# problem = tsplib95.load('ftv70.atsp')


# # convert into a networkx.Graph
# down = problem.get_graph()

# # convert into a numpy distance matrix
# hold1 = networkx.to_numpy_matrix(down)
# hold2 = np.array(hold1)

# datre=Dread(hold2)


#Insert title of data set here
holddat=np.array(pd.read_csv('29cities.txt', header=None, sep='\s+', engine='python'))
datre=Dread(holddat)

#User needs to replace checkopt with the optimal solution if using their own data set
#FIX THIS
checkopt=dataopt(len(datre))
print(checkopt)



#Insert desired number of trials in for "rarnge"
rarnge=20
#Insert desired Opt algorithm. "0" for 2-Opt, "1" for 3-Opt, "2" for k-Opt
typer=1
#Insert "1" for randomized tours, "0" for standard tour of (0,1,2,...)
rando=1
#Insert "1" to compare results for Opt+CCP+Opt and "0" for Opt+CCP
ktyper=0
#Insert "1" to compare results gap closure and "0" for relative improvement
optgap=0
#Insert "1" for success rate subroutine and "0" for Relative/Gap Closure Improvement 
succop=0
#Insert "1" if you want average and mean of subtours "0" otherwise
avsub=1




if succop ==1: 
    success = SuccessRate(checkopt,datre,rarnge,typer,rando,ktyper)
    if typer==0:
        print("For 2-Opt")
    if typer==1:
        print("For 3-Opt")
    if typer==2:
        print("For k-Opt")

    if ktyper==0:
        print("Success rate of CCP is",success)
    if ktyper==1:
        print("Success rate of CCP+k-Opt is",success)
else:
    ImproveRateU(checkopt,datre,rarnge,typer,rando,ktyper, optgap,avsub)


27603


KeyboardInterrupt: 

# Run This First

In [33]:

#The following function contains Optimal/Best found cost of tours
def dataopt(lenopt):
    if lenopt==12:
        return 1733
    if lenopt==22:
        return 781
    if lenopt==29:
        return 27603
    if lenopt==30:
        return 503
    if lenopt==59:
        return 992
    if lenopt==128:
        return 19033
    if lenopt==131:
        return 564
    if lenopt==237:
        return 1019
    if lenopt==343:
        return 1368
    if lenopt==53:
        return 6905
    if lenopt==34:
        return 1286
    if lenopt==36:
        return 1473
    if lenopt==39:
        return 1530
    if lenopt==125:
        return 36230
    if lenopt==171:
        return 2755
    if lenopt==71:
        return 38673
    if lenopt==70:
        return 1950
    if lenopt==45:
        return 1613 
    
    
    
#Used to determine success rate
def SuccessRate(opt,datsu,numtri,ktype,rands,kimp):
    checkop=opt
    
    #initialize expensive tour
    exptour=10000000

    sus1=0
    sus2=0
    sus3=0
    sus4=0
    sus5=0
    sus6=0
    suf1=0
    suf2=0
    Tour4s=[i for i in range(len(datsu))]
    Tour4s1=[i for i in range(len(datsu))]
    Tour3s=[i for i in range(len(datsu))]
    Tour3s1=[i for i in range(len(datsu))]
    Tour3s1u=[i for i in range(len(datsu))]
    Tour2s=[i for i in range(len(datsu))]
    Tour2s1=[i for i in range(len(datsu))]
    Tour5u=[i for i in range(len(datsu))]
    Cost2s=math.inf
    CCTour2s=math.inf
    CCTour2s1f=math.inf
    CCTour3s=math.inf
    CCTour4s=math.inf
    CCTour2s1=math.inf
    CCTour3s1=math.inf
    CCTour4s1=math.inf
    opt3hold=[0 for i in range(len(datsu))]
    tt=0
    tot1=0
    
    
    for k in range(numtri):
        start1=time.time()
        Datstoru =[]
        Datstoru = np.copy(datsu)
        routes=[]
        routes=[i for i in range(len(datsu))]
        #randomize tours
        if rands!=0:    
            random.shuffle(routes)
        #initialize tours
        if ktype==0:
            Tour2s, Cost2s = two_opt(datsu,routes)
            #print("HEEEEEEE",Tour2s)
        if ktype==1:
            Tour3s = tsp_3_opt(datsu,routes)
        if ktype==2:
            Tour2u, Cost2u = two_opt(datsu,routes)
            Tour5u = tsp_3_opt(datsu,Tour2u)
        
        
        if kimp==1:
            #Ensure that the Optimal solution isn't already found
            if cost(datsu,Tour3s)==checkop or cost(datsu,Tour2s)==checkop or cost(datsu,Tour5u)==checkop:
                k=k-1
            else:
                #subroutine for CCP+Opt
                if ktype==0:
                    CCTour2s1,CT2s1,h1,h2,tt = CCP(datsu,Tour2s)
                    Tour2s1, Cost2s1 = two_opt(datsu,CT2s1)
                if ktype==1:
                    CCTour3s1,CT3s1,h1,h2,tt = CCP(datsu,Tour3s)
                    for i in range(len(CT3s1)):
                        opt3hold[i]=CT3s1[i]
                    Tour3s1 = tsp_3_opt(datsu,opt3hold)
                if ktype==2:
                    CCTour4s1,CT4s1,h1,h2,tt= CCP(datsu,Tour5u)
                    for i in range(len(CT4s1)):
                        opt3hold[i]=CT4s1[i]
                    Tour4s1u, Cost2s1u = two_opt(datsu, opt3hold)
                    Tour3s1u = tsp_3_opt(datsu,Tour4s1u)
                
                #stores success rates
                if cost(datsu,Tour3s1) < cost(datsu,Tour3s):
                    sus5=sus5+1
                if CCTour2s1 < Cost2s:
                    sus4=sus4+1
                if CCTour2s1f < Cost2s:
                    suf1=suf1+1
                if CCTour2s1f < CCTour2s1:
                    suf2=suf2+1
                if cost(datsu,Tour3s1u) < cost(datsu,Tour5u):
                    sus6=sus6+1
                end1=time.time()
                
                tot1=tot1+end1-start1
                  
        #subroutine for CCP only        
        if kimp==0:
            if cost(datsu,Tour3s)==checkop or cost(datsu,Tour2s)==checkop:
                k=k-1
            else:
                if ktype==1:
                    CCTour3s,CT3s,h1,h2,tt = CCP(datsu,Tour3s)
                if ktype==0:
                    CCTour2s,CT2s,h1,h2,tt = CCP(datsu,Tour2s)
                if ktype==2:
                    CCTour4s,CT4s,h1,h2,tt = CCP(datsu,Tour5u)
                    
                #stores success rates
                if CCTour3s < cost(datsu,Tour3s):
                    sus2=sus2+1
                if CCTour2s < Cost2s:
                    sus1=sus1+1
                if CCTour4s < cost(datsu,Tour5u):
                    sus3=sus3+1
                end1=time.time()
                tot1=tot1+end1-start1
                    
        datsu = np.copy(Datstoru)
        print("time1",end1-start1)
            
    print("Total Time Running Time",tot1)
    if kimp==0:
        if ktype==0:
            return sus1
        if ktype==1:
            return sus2
        if ktype==2:
            return sus3

    if kimp==1:
        if ktype==0:
            return sus4
        if ktype==1:
            return sus5
        if ktype==2:
            return sus6
                    
        
#Used to calculate relative improvement and gap closure       
def ImproveRateU(optr,datsur,numtrir,ktyper,randsr,kimpr, optrel,avsub1):
    
    #variable initialization
    k=0
    sup1=0
    sup2=0
    totsu1=0
    totsu2=0
    totsu3=0
    totsu4=0
    here=0
    count1=0
    subs=0
    totsubs=0
    subco=0
    subarr=[]
    cly=0
    clarr=[]
    dondub=0
    opt3hold=[0 for i in range(len(datsur))]
    tt=0

    #Runs through a while loop until the specified number of trials is reached
    while k != numtrir:
        su1=0
        su2=0
        su3=0
        su4=0
        Datstorur =[]
        Datstorur = np.copy(datsur)
        route=[]
        route=[i for i in range(len(datsur))]
        random.shuffle(route)
    
        #Initial run for Opt algorithms
        if ktyper==0:
            Tour2, Cost2 = two_opt(datsur,route)
            cly=0
    
            if cost(datsur,route)==optr:
                k=k-1
            else:
                CCTour2,CT2,subs,cly,tt = CCP(datre,route)
                if CCTour2 == -1 or tt==1:
                    k=k-1
                    #If subtours exist
                else:
                    if cly!=0:
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                        clarr.append(cly)

                    Tour6,Cost6 = two_opt(datsur,CT2)
                    #calculate relative costs
                    su1=(Cost2-CCTour2)/(Cost2)            
                    su2=(Cost2-Cost6)/(Cost2) 
                    #calculate gap closure
                    su3=(Cost2-CCTour2)/(Cost2-checkopt)            
                    su4=(Cost2-Cost6)/(Cost2-checkopt) 
        
                    #add to total cost if improvement is made
                    if CCTour2 < Cost2 and Cost6 < Cost2 and checkopt < Cost2:
                        sup1=sup1+1
                        totsu1=totsu1+su1   
                        totsu2=totsu2+su2
                        totsu3=totsu3+su3   
                        totsu4=totsu4+su4
                    else:
                        k=k-1
            
            datsur = np.copy(Datstorur)
        
        if ktyper==1:
            Tour3 = tsp_3_opt(datsur,route)
            if cost(datre,Tour3)==checkopt:
                k=k-1
            else:
                CCTour3,CT3,subs,cly,tt = CCP(datsur,Tour3)
                if CCTour3 == -1 or tt==1:
                    k=k-1
                else:
                    if cly!=0:
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                        clarr.append(cly)
                
                    for i in range(len(CT3)):
                        opt3hold[i]=CT3[i]
                    
                    Tour3U = tsp_3_opt(datsur,opt3hold)
        
                    su1=(cost(datsur,Tour3)-CCTour3)/(cost(datsur,Tour3))
                    su2=(cost(datsur,Tour3)-cost(datre,Tour3U))/(cost(datsur,Tour3))
                    su3=(cost(datsur,Tour3)-CCTour3)/(cost(datsur,Tour3)-checkopt)
                    su4=(cost(datsur,Tour3)-cost(datsur,Tour3U))/(cost(datsur,Tour3)-checkopt)
      
                #total up improvement rates
                    if CCTour3 < cost(datsur,Tour3) and cost(datre,Tour3U) != CCTour3 and checkopt< cost(datre,Tour3):
                        totsu1=totsu1+su1   
                        totsu2=totsu2+su2
                        totsu3=totsu3+su3   
                        totsu4=totsu4+su4
                    else:
                        k=k-1

           
            datsur = np.copy(Datstorur)
            
        if ktyper==2:
            Tour2, Cost2 = two_opt(datsur,route)
            Tour3 = tsp_3_opt(datsur,Tour2)
            if cost(datsur,Tour3)==checkopt:
                k=k-1
            else:
                CCTour3,CT3,subs,cly,tt = CCP(datsur,Tour3)
                if CCTour3 == -1 or tt==1:
                    k=k-1
                else:
                    if cly!=0:
                        subarr.append(subs)
                        totsubs=subs+totsubs
                        subco=subco+1
                        clarr.append(cly)
                    
                    for i in range(len(CT3)):
                        opt3hold[i]=CT3[i]
                    Tour6,Cost6 = two_opt(datsur,opt3hold)
                    Tour3U = tsp_3_opt(datsur,Tour6)

                    su1=(cost(datsur,Tour3)-CCTour3)/(cost(datsur,Tour3))
                    su2=(cost(datsur,Tour3)-cost(datsur,Tour3U))/(cost(datsur,Tour3))               
                    su3=(cost(datsur,Tour3)-CCTour3)/(cost(datsur,Tour3)-checkopt)           
                    su4=(cost(datsur,Tour3)-cost(datsur,Tour3U))/(cost(datsur,Tour3)-checkopt)
        
                    if CCTour3 < cost(datre,Tour3) and cost(datre,Tour3U) < CCTour3 and checkopt < cost(datre,Tour3):
                        totsu1=totsu1+su1   
                        totsu2=totsu2+su2
                        totsu3=totsu3+su3   
                        totsu4=totsu4+su4
                        print("new",cost(datsur,Tour3U),"old", cost(datsur,Tour3))
                    else:
                        k=k-1
                    
        if avsub1 == 1:
            if subco==numtrir-1 and dondub==0:
                dondub=1
                #print("average",statistics.mean(clarr),"subco")
                #print("Median", statistics.median(clarr), "Range", min(clarr),"-",max(clarr))

        k=k+1        
        datsur = np.copy(Datstorur)
           
    if ktyper==0:
        print("Success 2+CCP, Rel",totsu1/rarnge)     
        print("Success 2+CCP+2, Rel",totsu2/rarnge)
        print("Success 2+CCP, OptGap",totsu3/rarnge)     
        print("Success 2+CCP+2, OptGap",totsu4/rarnge)
        
    if ktyper==1:
        print("Success 3+CCP",totsu1/rarnge)     
        print("Success 3+CCP+3",totsu2/rarnge)
        print("Success 3+CCP Gap",totsu3/rarnge)     
        print("Success 3+CCP+3 Gap",totsu4/rarnge)
        
    if ktyper==2:
        print("Success k+CCP",totsu1/rarnge)     
        print("Success k+CCP+k",totsu2/rarnge)
        print("Success k+CCP Gap",totsu3/rarnge)     
        print("Success k+CCP+k Gap",totsu4/rarnge)
        
        


def CycleBreakSub(ListBruk, jumpk, revk):
    #create a 2-D array to store a copy of jumpk
    jumpystork=[[-1 for i in range(2)] for j in range(len(jumpk))]
    for i in range(len(jumpk)):
        for j in range(2):
            jumpystork[i][j]=jumpk[i][j]
    UpdatedLuk=[-1]
    UpdatedFun=[]
    indyx=0
    countroo=-1
    minny=math.inf
    minnystore=math.inf
    jumpystore=math.inf
    jumpnex=-1
    infcounter=0
    hhold=-1
    zerond=-1
    brucker=-1
    kstore=-1
    jumpzer=-1
    minicho=-1
    stmin=-1
    jmin=-1
    
    
    #jumpzer is set to "0" because it can cause issues sometimes
    for i in range(len(jumpk)):
        if jumpk[i][0] == 0:
            jumpzer=0
        
    #continue to run until negative cycle has been iterated through entirely
    #process is a little convaluted, but essentially it considers every possible case
    #that could appear for a given "edge" and prepares them for the next loop
    while len(UpdatedLuk) != len(ListBruk):
        if UpdatedLuk[0] == -1:
            UpdatedLuk=[]
        for i in range(len(jumpk)):
            minny=revk[i][0]
            if minny < minnystore and minny>=jumpnex:
                minnystore=minny
        if (stmin==minnystore and jmin==jumpnex):
            break
        stmin=minnystore
        jmin=jumpnex
        if minnystore==jumpnex:
            if ListBruk[indyx] not in UpdatedLuk:
                UpdatedLuk.append(minnystore)
            else:
                break
        while indyx != minnystore+1:
            if indyx==len(ListBruk):
                break
            if ListBruk[indyx]==0 and zerond==0:
                brucker=0
                break
            if ListBruk[indyx]==0:
                zerond=0
            if ListBruk[indyx] not in UpdatedLuk:
                UpdatedLuk.append(ListBruk[indyx])
                countroo=countroo+1
            indyx=indyx+1
        if brucker==0 and jumpzer==-1:
            break
        for h in range(len(revk)):
            if(jumpk[h][0]==minnystore):
                jumpnex=jumpk[h][1]
                indyx=jumpnex
                jumpk[h][0]=math.inf
                infcounter=infcounter+1
        minny=math.inf
        minnystore=math.inf
    
    #print("updatedFu",UpdatedFun)
    UpdatedFun.append(UpdatedLuk)
    UpdatedLuk=[]
    jumpnex=math.inf
    minny=-1
    startu=0
    startval=-1
    indyx=0
    hstore=-1
    istore=-1
    minny=math.inf
    minnystore=math.inf
    jumpy=math.inf
    jumpystore=math.inf
    jumpnex=0
    countroo=0
    infcounter=1
    finishd=0
    brek=0
    
    #This is where the CCTP sends a "flow of one" through the network
    while infcounter < len(revk):
        
        for h in range(len(revk)):
            if(math.isinf(jumpk[h][0])==0):
                jumpy=jumpk[h][1]
                if jumpy < jumpystore and minny>jumpnex:
                    jumpystore=jumpy
                    hstore=h
                    
               
                if(revk[i][0]<minnystore and minny>=jumpnex):
                        minny=revk[i][0]
                        istore=i
                        
        infcounter=infcounter+1 
        for i in range(len(jumpk)):
            minny=revk[i][0]
            if minny < minnystore and minny >= jumpystore: 
                minnystore=minny
        indyx=jumpystore
        minny=math.inf
        if jumpystore != minnystore +1:       
            while countroo + jumpystore != minnystore +1:
                if indyx==minnystore+1 and UpdatedLuk:
                    if UpdatedLuk[len(UpdatedLuk)-1] != UpdatedFun[len(UpdatedFun)-1][len(UpdatedFun[len(UpdatedFun)-1])-1]:
                        UpdatedFun.append(UpdatedLuk)
                    UpdatedLuk=[]
                    break
                if indyx == len(ListBruk):
                    break
                if ListBruk[indyx] not in UpdatedLuk:
                    UpdatedLuk.append(ListBruk[indyx])
                countroo=countroo+1
                indyx=indyx+1 
            
            for i in range(len(jumpk)):
                if jumpystork[i][0]+1==indyx:
                    if UpdatedLuk:
                        if UpdatedLuk[0] == jumpystork[i][1]:
                            UpdatedFun.append(UpdatedLuk)
                            UpdatedLuk=[]
                            break
        countroo=0
            
        jumpk[hstore][0]=math.inf
        hstore=-1
        istore=-1
        countroo=countroo-1
        for h in range(len(revk)):
            if(jumpk[h][0]==minnystore):
                jumpnex=jumpk[h][1]
                indyx=jumpnex
        minny=math.inf
        minnystore=math.inf
        jumpy=math.inf
        jumpystore=math.inf
        
    return UpdatedFun


def subcon(routco,Dot):
    for j in range(len(routco)):
        for i in range(len(routco[j])):
            if routco[j][i]>(len(Dot)-1):
                routco[j][i]=routco[j][i]-len(Dot)
    return routco

#returns the indices of each subtour
def subindex(subin,Dotin,rott):
    indiid=[]
    indiisto=[]
    
    for k in range(len(subin)):
        for i in range(len(subin[k])):
            for j in range(len(Dotin)):
                if subin[k][i]==rott[j]:
                    indiid.append(j)
        indiisto.append(indiid)
        indiid=[]
    
    return indiisto


def CCP(Datc,routcc):
    
    copyroot=routcc.copy()
    routind=[i for i in range(len(routcc))]
    Datstor =[]
    Datstor = np.copy(Datc)
    coststory=[]
    Plourd=[]
    Plourdhold=[]
    coutn=1
    coutnn=1
    costup=math.inf
    recyc=[]
    emptour=[]
    subnumb=0
    lyc=0
    costmax=math.inf
    circheck=0
    tc=0
        
    #Form transformed distance matrix
    Dorre =adjusted(Datc,routcc)
    #Form a node-arc incident matrix, 
    #set of edges, cost of edges, transformed matrix
    nodeorce,boe,objee,Dor=routesform(Dorre,routcc)
    #use pyomo and matrices from previous function 
    #to find min-cost circulation
    subbe=circulate(nodeorce,boe,objee,Dor)

    #If there aren't any subtours, your tour is already improved, return it
    if len(subbe)==0:
        costup=cost(Datc,routcc)
        return costup, routcc, subnumb, lyc, tc

    lyc=len(subbe)
    jumpsto=[]
    revsto=[]
    
    #Converts subtours from 2n x 2n matrix to n x n matrix
    subber=subcon(subbe,Datc)
    #returns the indices of each subtour
    indii=subindex(subber,Datc,routcc)
    
    #checks for special case for a certain set of indices
    #that cuases issues
    for i in range(len(indii)):
        for j in range(int(len(indii[i])/2)-1):
            if indii[i][2*j]==(indii[i][2*j+1]-1) or (indii[i][2*j]==0 and (indii[i][2*j+1]==len(Datc)-1)):
                circhek=1
    
    
    #jumpc stores "edges" that add flow, revc are reversals
    for j in range(len(subber)):
        jumpc,revc = Jumps(routcc,indii[j])
        for i in range(len(jumpc)):
            jumpsto.append(jumpc[i])
            revsto.append(revc[i])
            
    
    jumpcopy=jumpsto.copy()
            
    #Subtourc contains the set of subtours that exist on the network prior to any patching
    Subtourc= CycleBreakSub(routind,jumpsto,revsto)
            
    #Puts together subtours into a single array
    Listfin=Listy(routcc,Subtourc)   
        
    #The following subroutine is the "Adjusted Patching Algorithm"
    if len(Listfin) > 1:
        #Tourpr is a function which contains two functions that patch subtours together
        Plourd=Tourpr(Listfin,Datc,Subtourc)
        if costd(Datc,Plourd) not in coststory:
            coststory.append(costd(Datc,Plourd))
            coutn=coutn+1
            #Included check to make sure the final tour is actually a tour
            tc=Tourcheck(Plourd,Datc)
                
        if costd(Datc,Plourd)<costmax:
            Plourdhold=np.copy(Plourd)
            costmax=costd(Datc,Plourd)
                
    else:
        if costd(Datc,Listfin[0]) not in coststory:
            coststory.append(costd(Datc,Listfin[0]))
            coutn=coutn+1
            Tourcheck(Listfin[0],Datc)
            Plourd=Listfin[0]
                
    
    for i in range(len(coststory)):
        if coststory[i] < costup:
            costup=coststory[i]
    costup=costd(Datc,Plourd)

    if len(Plourdhold) == 0:
        return cost(Datc,copyroot), copyroot, subnumb, lyc, tc
    
    if circheck==1:
        costmax=-1
    
    return costmax, Plourdhold, subnumb, lyc, tc
        

def Tourcheck(Trch,Tlen):
    check1=0
    check2=0
    Trr=Trch.copy()
    good=0
    if len(Trch)==len(Tlen):
        check1=1
    
    for i in range(len(Trch)):
        if i not in Trr:
            break
        Trr.remove(i)
        
    if not Trr:
        check2=1
        
    if check1 == 1 and check2 ==1:
        good=0
    else:
        print("not a tour")
        good=1
        
    return good
        
        
        

def Dread(matri):

    int1=int(len(matri))
    int2=int(len(matri[0]))
    int3=int(int1*int2)
    msize=len(matri[0])
    ares = np.array([ 0 for i in range(int3)])
    aresp = np.array([ [ 0 for i in range(msize) ] for j in range(msize) ])
    d=0
    mo=-1

    for i in range(int1):
        for j in range(int2):
            if (np.isnan(matri[i][j])):
                d=d+1
                break
            mo=mo+1
            ares[mo]=matri[i][j]

    mo=-1

    for i in range(msize):
        for j in range(msize):
            mo=mo+1
            aresp[i][j]=ares[mo]

    return aresp

def cost(costmatt, routeee):
    sumr=0
    sumr=costmatt[int(routeee[int(len(routeee))-1])][int(routeee[0])]
    for i in range(len(routeee)-1):
        sumr += costmatt[routeee[i]][routeee[i+1]]
    return sumr


def costd(cost_mato, routo):
    sumro=0
    sumro=cost_mato[routo[len(routo)-1]][routo[0]]
    for i in range(len(routo)-1):
        sumro += cost_mato[routo[i]][routo[i+1]]
    return sumro 

def adjustedcir(cost_maj,routj):
    
    sizec=int(len(cost_maj[0]))
    
    #BEGIN Matrix Initialization Process
    
    Pstore= np.array([ [ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec) ] for k in range (2*sizec)])  
    Dstore= np.array([ [ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec) ] for k in range (2*sizec)])
    
    D = np.array([ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec) ])
    P = np.array([ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec)])    

    
    for q in range(sizec):
        for w in range(sizec):
            if(w != q):
                D[q+sizec][w]=cost_maj[q][w]
                P[q+sizec][w]=q+sizec
                
        
    for i in range(len(routj)-1):  
        D[routj[i]+sizec][routj[i+1]]=math.inf
        D[routj[i+1]+sizec][routj[i]]=math.inf
        D[routj[i+1]][routj[i]+sizec]=-cost_maj[routj[i]][routj[i+1]]
        P[routj[i]+sizec][routj[i+1]]=math.inf
        P[routj[i+1]+sizec][routj[i]]=math.inf
        P[routj[i+1]][routj[i]+sizec]=routj[i+1]
        
    D[routj[0]][routj[len(routj)-1]+sizec]=-cost_maj[routj[len(routj)-1]][routj[0]]
    P[routj[0]][routj[len(routj)-1]+sizec]=routj[0]
    D[routj[0]+sizec][routj[len(routj)-1]]= math.inf
    D[routj[len(routj)-1]+sizec][routj[0]] = math.inf
    P[routj[0]+sizec][routj[len(routj)-1]]= math.inf
    P[routj[len(routj)-1]+sizec][routj[0]] = math.inf
    
    return D


def adjusted(cost_maj,routj):
    
    sizec=int(len(cost_maj[0]))
    
    #BEGIN Matrix Initialization Process
    
    Pstore= np.array([ [ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec) ] for k in range (2*sizec)])  
    Dstore= np.array([ [ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec) ] for k in range (2*sizec)])
    
    D = np.array([ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec) ])
    P = np.array([ [ (math.inf) for i in range(2*sizec) ] for j in range(2*sizec)])    

    
    for q in range(sizec):
        for w in range(sizec):
            if(w != q):
                D[q+sizec][w]=cost_maj[q][w]
                P[q+sizec][w]=q+sizec
                
        
    for i in range(len(routj)-1):  
        D[routj[i]+sizec][routj[i+1]]=math.inf
        D[routj[i+1]+sizec][routj[i]]=math.inf
        D[routj[i+1]][routj[i]+sizec]=-cost_maj[routj[i]][routj[i+1]]
        P[routj[i]+sizec][routj[i+1]]=math.inf
        P[routj[i+1]+sizec][routj[i]]=math.inf
        P[routj[i+1]][routj[i]+sizec]=routj[i+1]
        
    D[routj[0]][routj[len(routj)-1]+sizec]=-cost_maj[routj[len(routj)-1]][routj[0]]
    P[routj[0]][routj[len(routj)-1]+sizec]=routj[0]
    D[routj[0]+sizec][routj[len(routj)-1]]= math.inf
    D[routj[len(routj)-1]+sizec][routj[0]] = math.inf
    P[routj[0]+sizec][routj[len(routj)-1]]= math.inf
    P[routj[len(routj)-1]+sizec][routj[0]] = math.inf
    
    #END Matrix Initialization Process
    
    #Begin Floyd-Warshall Algorithm
    for k in range(2*sizec):
        for i in range(2*sizec):
            for j in range(2*sizec):
                if i != k and j != k and i != (k+sizec) and j != (k+sizec):
                    if D[i,j] > D[i,k] + D[k,j]:
                        P[i,j]=P[k,j]
                        Pstore[k]=P
                        D[i,j] = D[i,k] + D[k,j]
                        Dstore[k]=D

    return D, P, Pstore, Dstore




def transformeD(routD,cost_matD):
    size=len(cost_matD)
    Dn = np.array([ [ (math.inf) for i in range(2*size) ] for j in range(2*size) ])
    
    for i in range(size):
        for j in range(size):
            Dn[size+i,j] = cost_matD[i][j]
            Dn[i,size+j] = cost_matD[i][j]
            Dn[i,j] = cost_matD[i][j]
            Dn[i,j] = cost_matD[i][j]
            
    for i in range(len(routD)-1):
        Dn[routD[i+1]][routD[i]+size]=-cost_matD[routD[i]][routD[i+1]]
        Dn[routD[i+1]+size][routD[i]]=-cost_matD[routD[i]][routD[i+1]]
        Dn[routD[i+1]+size][routD[i]+size]=-cost_matD[routD[i]][routD[i+1]]
        Dn[routD[i+1]][routD[i]]=-cost_matD[routD[i]][routD[i+1]]
    
       
    Dn[routD[0]][routD[len(routD)-1]+size]=-cost_matD[routD[len(routD)-1]][routD[0]]
    Dn[routD[0]+size][routD[len(routD)-1]]=-cost_matD[routD[len(routD)-1]][routD[0]]
    Dn[routD[len(routD)-1]+size][routD[0]]=-cost_matD[routD[len(routD)-1]][routD[0]]
    Dn[routD[0]][routD[len(routD)-1]]=-cost_matD[routD[len(routD)-1]][routD[0]]
    Dn[routD[len(routD)-1]][routD[0]+size]=-cost_matD[routD[len(routD)-1]][routD[0]]
    
    
    return Dn


def translpreD(cost_matL,ListL):
    newLL=[]
    for i in range(len(ListL)):
        if int(ListL[i]) > (len(cost_matL)-1):
            newLL.append(int(ListL[i]-len(cost_matL)))
        else:
            newLL.append(ListL[i])
    
    return newLL    
    

#The following function is allows for negative costs to be added to a final tour
def costneg(cost_matneg, routneg):
    sumreg=0
    sumreg= cost_matneg[routneg[len(routneg)-1]][routneg[0]]   
    for i in range(len(routneg)-1):
        sumreg+=cost_matneg[routneg[i]][routneg[i+1]]
    return sumreg

def pred(Ped, ved):
    
    Led=[]
    ked=int(ved)
    ied=int(ved)
    count=-1
    counter=0
    checkr=0
                   
    if math.isinf(Ped[ied,ked]) != 1:
        while (int(Ped[ied,ked])!= ied and math.isinf(Ped[ied,ked]) != 1):
            ked=int(Ped[ied,ked])
            count=count+1
            while counter < count:
                if Led[counter] == ked: 
                    while Led[checkr] != ked:
                        Led.remove(Led[checkr])
                    return(Led)
                counter=counter+1
                
            counter=0
            Led.append(ked)
            if math.isinf(Ped[ied,ked]) == 1:
                Led=[]
                break
        
    Led.append(ved)
    return(Led)

def Preder(Dored, Pored, routro,datarr):
    iind=2
    yup=0
    indexx=[]
    indor=[]
    indes=[]
    nestore=[]
    costore=[]
    iindstord=[]
    routerind = [[-1 for i in range(len(datarr))] for j in range(iind)]
    
    #Matrix Transformation Step
    Dt=transformeD(routro,datarr)
    LiStore=routro
    countind=0

    #Form a 2-D matrix to store indices of negative cycle
    for i in range(iind):
        for j in range(len(datarr)):
            routerind[0][j]=routro[j]
            routerind[1][j]=j
            
    #runs through the various stored "D" matrices to find negative cycles
    for i in range(len(routro)-1):
        for j in range(2*len(routro)-1):
            List=pred(Pored[j], i)
            List.reverse()   

            if (Dored[j][i][i]<0 and len(List)>1): 
                #translates indices of tranformed matrix into indices of original graph
                Li=translpreD(datarr,List)
                for p in range(len(Li)):
                    while Li[p] != routerind[0][countind]:
                        countind=countind+1
                    iindstord.append(routerind[1][countind])
                    countind=0
                for p in range(len(indexx)):
                    if indexx[p][1] == iindstord:
                        yup=1
                if yup == 0:
                    indes.append(Li)
                    indes.append(iindstord)
                    indes.append(costneg(Dt,Li))
                    indexx.append(indes)
                yup=0
                indes=[]
                iindstord=[]
                
    return indexx, routerind




def Jumps(ListBru,inDru):
    newLBru=[[-1 for i in range(len(ListBru))]]
    Listhold=[]
    reverses=[]
    revstor=[-1,-1]
    jumps=[]
    jumpstor=[-1,-1]
    indexhold=-1
    counteu=0
    
    
    #
    if (inDru[len(inDru)-1]==inDru[0]+1 or (inDru[len(inDru)-1]==0 and inDru[0]==(len(ListBru)-1))):
            revstor[0]=inDru[len(inDru)-1]
            revstor[1]=inDru[0]
            reverses.append(revstor)
            revstor=[-1 for i in range(2)]
            jumpstor[0]=inDru[0]
            jumpstor[1]=inDru[1]
            jumps.append(jumpstor)
            jumpstor=[-1 for i in range(2)]
            
            
    if inDru[len(inDru)-2]==inDru[len(inDru)-1]+1 or inDru[len(inDru)-2]==0:
            revstor[0]=inDru[len(inDru)-1]
            revstor[1]=inDru[len(inDru)-2]
            reverses.append(revstor)
            revstor=[-1 for i in range(2)]
            jumpstor[0]=inDru[len(inDru)-1]
            jumpstor[1]=inDru[0]
            jumps.append(jumpstor)
            jumpstor=[-1 for i in range(2)]
    
    for p in range(len(inDru)-2):
        if inDru[p]==inDru[p+1]+1 or (inDru[p]+len(ListBru)-1==inDru[p+1]):
            revstor[0]=inDru[p+1]
            revstor[1]=inDru[p]
            reverses.append(revstor)
            revstor=[-1 for i in range(2)]
            jumpstor[0]=inDru[p+1]
            jumpstor[1]=inDru[p+2]
            jumps.append(jumpstor)
            jumpstor=[-1 for i in range(2)]
            
            
    #print("jumps in function",jumps)
    
    return jumps,reverses
            

def Listy(rourt, Subby):

    ListS=[]
    Listfiny=[]
    for k in range(len(Subby)):
        for j in range(len(Subby[k])):
            ListS.append(rourt[Subby[k][j]])
        Listfiny.append(ListS)
        ListS=[]

    return Listfiny



def Patcher(cycle1, cycle2,cost_matPat):
        
    store2=math.inf
    patch=[]
    
    if (len(cycle1)==1):
        for i in range(len(cycle2)-1):
            store1=cost_matPat[cycle2[i]][cycle1[0]]+cost_matPat[cycle1[0]][cycle2[i+1]]-cost_matPat[cycle2[i]][cycle2[i+1]]
            if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle2[i+1])
                patch.append(cycle2[i])
                patch.append(cycle1[0])
        
    if (len(cycle2)==1):
        for i in range(len(cycle1)-1):
            store1=cost_matPat[cycle1[i]][cycle2[0]]+cost_matPat[cycle2[0]][cycle1[i+1]]-cost_matPat[cycle1[i]][cycle1[i+1]]
            if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[i+1])
                patch.append(cycle1[i])
                patch.append(cycle2[0])
        
    for i in range(len(cycle1)-1):
        for j in range(len(cycle2)-1):
            store1=cost_matPat[cycle1[i]][cycle2[j+1]]+cost_matPat[cycle2[j]][cycle1[i+1]]-cost_matPat[cycle1[i]][cycle1[i+1]]-cost_matPat[cycle2[j]][cycle2[j+1]]
            if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[i+1])
                patch.append(cycle1[i])
                patch.append(cycle2[j+1])
                patch.append(cycle2[j])
                                        
    for i in range(len(cycle1)-1):
        store1=cost_matPat[cycle1[i]][cycle2[0]]+cost_matPat[cycle2[len(cycle2)-1]][cycle1[i+1]]-cost_matPat[cycle1[i]][cycle1[i+1]]-cost_matPat[cycle2[len(cycle2)-1]][cycle2[0]]
        if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[i+1])
                patch.append(cycle1[i])
                patch.append(cycle2[0])
                patch.append(cycle2[len(cycle2)-1])
                
    for j in range(len(cycle2)-1):
        store1=cost_matPat[cycle1[len(cycle1)-1]][cycle2[j+1]]+cost_matPat[cycle2[j]][cycle1[0]]-cost_matPat[cycle1[len(cycle1)-1]][cycle1[0]]-cost_matPat[cycle2[j]][cycle2[j+1]]
        if store1<store2:
                store2=store1
                patch.clear()
                patch.append(cycle1[0])
                patch.append(cycle1[len(cycle1)-1])
                patch.append(cycle2[j+1])
                patch.append(cycle2[j])
                
    store1=cost_matPat[cycle1[len(cycle1)-1]][cycle2[0]]+cost_matPat[cycle2[len(cycle2)-1]][cycle1[0]]-cost_matPat[cycle1[len(cycle1)-1]][cycle1[0]]-cost_matPat[cycle2[len(cycle2)-1]][cycle2[0]]
    if store1<store2:
        store2=store1
        patch.clear()
        patch.append(cycle1[0])
        patch.append(cycle1[len(cycle1)-1])
        patch.append(cycle2[0])
        patch.append(cycle2[len(cycle2)-1])
        
                
    return store2, patch


def Patched(cycle1pa, cycle2pa, patchd):
            
    newLpa=[]
    newS=[]
    newE=[]
    newM1=[]
    newM2=[]
    switchpa=0
    firstpa=-1
    countrpa=0
    
    if len(patchd)==3:
        
        if len(cycle1pa)==1:
            for i in range(len(cycle2pa)):
                newLpa.append(cycle2pa[i])
                if cycle2pa[i]==patchd[1]:
                    newLpa.append(cycle1pa[0])
            return newLpa
                    
            
        if len(cycle2pa)==1:
            for i in range(len(cycle1pa)):
                newLpa.append(cycle1pa[i])
                if cycle1pa[i]==patchd[1]:
                    newLpa.append(cycle2pa[0])
            return newLpa           
                
    for i in range(len(cycle1pa)):
        if switchpa==0:
            newS.append(cycle1pa[i])
            if cycle1pa[i]==patchd[1]:
                switchpa=1
                firstpa=1
  
        if switchpa==1:
            if(firstpa != 1):
                newE.append(cycle1pa[i])
            firstpa=0

    switchpa=0
    for i in range(len(cycle2pa)):
        if switchpa==0:
            newM1.append(cycle2pa[i])
            if cycle2pa[i]==patchd[3]:
                switchpa=1
                firstpa=1
  
        if switchpa==1:
            if(firstpa != 1):
                newM2.append(cycle2pa[i])
            firstpa=0
            
    newLpa=newS+newM2+newM1+newE
    
    return newLpa



def Tourpr(ListFind,datad,Subtad):
    if len(ListFind) > 1:
        costPat,patchPat= Patcher(ListFind[0],ListFind[1],datad)
        Patour= Patched(ListFind[0], ListFind[1], patchPat)
        if len(ListFind) > 1:
            for j in range(len(Subtad)-2):
                costPat,patchPat = Patcher(Patour,ListFind[j+2],datad)
                Patour=Patched(Patour,ListFind[j+2],patchPat)
             
    return Patour


def circulate(nodeorc,bo,obje,Dorr):
    
    mart = nodeorc
    leng=len(nodeorc[0])
    rows=len(nodeorc)
    numbcon=[i for i in range(len(nodeorc))]
    bright=[0 for i in range(len(nodeorc))]
    testy=[1 for i in range(len(nodeorc[0]))]
    mat = [i for i in range(leng)]   
    counteri=0
    edgehold= [0 for i in range(2)]
    edgehee=[]
    suber1=[]
    suber2=[]

    #initialize variables
    data = {'INDEX': numbcon,
            'FLOW': bright}

    df = pd.DataFrame(data)
    df.set_index('INDEX', inplace=True)

    model = pe.ConcreteModel()
    #forms variables. Must be >= 0
    model.duid_bids = pe.Var(mat, domain=pe.PositiveReals)
    #objective function
    model.Objective   = Objective(expr = np.dot(obje, model.duid_bids), sense=minimize)

    #set of constraonts to ensure a net flow of zero
    #on each vertex
    def vol_constraint(m, date):
        expree=0
        for i in range(len(mart[date])):
            if mart[date][i]!=0:
                expree+=mart[date][i]*model.duid_bids[i]
        return expree <= df['FLOW'].loc[date]

    #max flow on each edge is one
    def vol_constraint2(m, date):
        return model.duid_bids[date] <= 1

    #uses previous to functions to form constraints
    model.vol_constraint = pe.Constraint(df.index, rule=vol_constraint)
    model.vol_constraint2 = pe.Constraint(mat, rule=vol_constraint2)

    #runs LP solver
    solver = pe.SolverFactory('glpk')
    status = solver.solve(model)
    
    #next two loops convert solution into edges
    for v in model.component_data_objects(Var):
        if(v.value==1):
            for j in range(len(bo[0])):
                if (bo[counteri][j] == -1):
                    edgehold[0]=j
                if (bo[counteri][j] == 1):
                    edgehold[1]=j
            edgehee.append(edgehold)
            edgehold= [0 for i in range(2)]
        counteri+=1
        
    while edgehee:
        suber1=[]
        suber1.append(edgehee[0][0])
        suber1.append(edgehee[0][1])
        edgehee.remove(edgehee[0])
        countdel=0
        for i in range(len(edgehee)-countdel):
            for j in range(len(edgehee)-countdel):
                if j < len(edgehee):
                    if suber1[len(suber1)-1]==edgehee[j][0]:
                        suber1.append(edgehee[j][1])
                        edgehee.remove(edgehee[j])
        suber2.append(suber1)
                
    #pops last edge to avoid repeats in solution
    for i in range(len(suber2)):
        suber2[i].pop(len(suber2[i])-1) 
    
    return suber2


def routesform(Datta,routesee):
    
    nodearc=[]
    Dor=adjustedcir(datre,routesee)
            
    bsto=[0 for _ in range(len(Dor))]
    b=[]
    obj=[]

    for i in range(len(Dor)):
        for j in range(len(Dor)):
            if np.isinf(Dor[i][j])==False:
                obj.append(Dor[i][j])
                bsto[i]=-1
                bsto[j]=1
                b.append(bsto)
                bsto=[0 for _ in range(len(Dor))]

    for i in range(len(b[0])):
        newmaty=[]
        for j in range(len(b)):  
            newmaty.append(b[j][i])
        nodearc.append(newmaty)
        
    return nodearc,b,obj,Dor
    


# Updated Opts

In [19]:

#the following opts are adjusted to allow for reversals
#and original functions were pulled from various sources
def two_opt(cost_matrt,routrt):
    routrhold=[]
    routre=[]
    routrhold=np.copy(routrt)
    best = routrt.copy()
    improved = True
    while improved:
        improved = False
        for i in range(1, len(routrt)-2):
            for j in range(i+1, len(routrt)+1):
                if j-i == 1: continue # changes nothing, skip then
                new_route = routrt.copy()
                new_route[i:j] = routrt[j-1:i-1:-1] # this is the 2woptSwap
                routre=list(reversed(new_route))
                if cost(cost_matrt, routre) < cost(cost_matrt, new_route):
                    new_route=routre
                if cost(cost_matrt, new_route) < cost(cost_matrt, best):
                    best=new_route
                    improved = True
        routrt = best
        
    #print("best",best,"hold",routrhold)
    if cost(cost_matrt, best) > cost(cost_matrt, routrhold):
        return routrhold, cost(cost_matrt, routrhold)
    else:
        return best, cost(cost_matrt, best)




class OptCase(Enum):
    opt_case_1 = "opt_case_1"
    opt_case_2 = "opt_case_2"
    opt_case_3 = "opt_case_3"
    opt_case_4 = "opt_case_4"
    opt_case_5 = "opt_case_5"
    opt_case_6 = "opt_case_6"
    opt_case_7 = "opt_case_7"
    opt_case_8 = "opt_case_8"
    opt_case_9 = "opt_case_9"
    opt_case_10 = "opt_case_10"
    opt_case_11 = "opt_case_11"
    opt_case_12 = "opt_case_12"
    opt_case_13 = "opt_case_13"
    opt_case_14 = "opt_case_14"
    opt_case_15 = "opt_case_15"
    opt_case_16 = "opt_case_16"
    
    

def tsp_3_opt(graph, route):
#     routeup=np.array(route)
    routeup=route
    route=[0 for i in range(len(routeup))]
    for i in range(len(routeup)):
        route[i]=routeup[i]
    import time
    #The following variable was included by me because this 3-opt can get stuck in an infinite loop
    timeout = time.time() + 2
    lengty=len(graph)
    moves_cost = {OptCase.opt_case_1: 0, OptCase.opt_case_2: 0,
                  OptCase.opt_case_3: 0, OptCase.opt_case_4: 0, OptCase.opt_case_5: 0,
                  OptCase.opt_case_6: 0, OptCase.opt_case_7: 0, OptCase.opt_case_8: 0,
                  OptCase.opt_case_9: 0, OptCase.opt_case_10: 0,
                  OptCase.opt_case_11: 0, OptCase.opt_case_12: 0, OptCase.opt_case_13: 0,
                  OptCase.opt_case_14: 0, OptCase.opt_case_15: 0, OptCase.opt_case_16: 0}
    improved = True
    best_found_route = route
    besty_route = route
    stored=cost(graph,best_found_route)
    timeout_start = time.time()
    while improved:
        improved = False
        for (i, j, k) in possible_segments(len(graph)):
            time27 = time.time()
            if time27 > timeout:
                return besty_route
            if(cost(graph,best_found_route)<stored):
                besty_route=best_found_route
            for opt_case in OptCase:
                moves_cost[opt_case] = get_solution_cost_change(graph, best_found_route, opt_case, i, j, k)
            # we need the minimum value of substraction of old route - new route
            best_return = max(moves_cost, key=moves_cost.get)
            if moves_cost[best_return] > 0:
                best_found_route = reverse_segments(best_found_route, best_return, i, j, k)
                improved = True
                break
    # just to start with the same node -> we will need to cycle the results.
    cycled = cycle(best_found_route)
    skipped = dropwhile(lambda x: x != 0, cycled)
    sliced = islice(skipped, None, len(best_found_route))
    best_found_route = list(sliced)
    return best_found_route


def possible_segments(N):
    """ Generate the combination of segments """
    segments = ((i, j, k) for i in range(N) for j in range(i + 2, N-1) for k in range(j + 2, N - 1 + (i > 0)))
    return segments


def get_solution_cost_change(graph, route, case, i, j, k):
    """ Compare current solution with 7 possible 3-opt moves"""
    A, B, C, D, E, F = route[i - 1], route[i], route[j - 1], route[j], route[k - 1], route[k % len(route)]
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]
    
    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        return 0
    elif case == OptCase.opt_case_2:
        # second case is the case A'BC
        return -cost(graph,list(reversed(first_segment)) + second_segment + third_segment)+cost(graph,route)
    elif case == OptCase.opt_case_3:
        # ABC'
        return -cost(graph,first_segment + second_segment + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_4:
        # A'BC'
        return -cost(graph,list(reversed(first_segment)) + second_segment + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_5:
        # A'B'C
        return -cost(graph,list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment)+cost(graph,route)
    elif case == OptCase.opt_case_6:
        # AB'C
        return -cost(graph,first_segment + list(reversed(second_segment)) + third_segment)+cost(graph,route)
    elif case == OptCase.opt_case_7:
        # AB'C'
        return -cost(graph,first_segment + list(reversed(second_segment)) + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_8:
        # A'B'C
        return -cost(graph,list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_9:
        # second case is the case A'BC
        return -cost(graph,list(reversed(third_segment)) + list(reversed(second_segment)) + first_segment)+cost(graph,route)
    elif case == OptCase.opt_case_10:
        # ABC'
        return -cost(graph,third_segment + list(reversed(second_segment)) + list(reversed(first_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_11:
        # A'BC'
        return -cost(graph,third_segment + second_segment + list(reversed(first_segment)))+cost(graph,route) 
    elif case == OptCase.opt_case_12:
        # A'B'C
        return -cost(graph,third_segment + second_segment + list(reversed(first_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_13:
        # AB'C
        return -cost(graph,list(reversed(third_segment)) + second_segment + list(reversed(first_segment)))+cost(graph,route)
    elif case == OptCase.opt_case_14:
        # AB'C'
        return -cost(graph,list(reversed(third_segment)) + second_segment + first_segment)+cost(graph,route)
    elif case == OptCase.opt_case_15:
        # A'B'C
        return -cost(graph,third_segment + second_segment + first_segment)+cost(graph,route)
    elif case == OptCase.opt_case_16:
        # A'B'C
        return -cost(graph,list(reversed(third_segment)) + list(reversed(second_segment)) + list(reversed(first_segment)))+cost(graph,route)

    

def reverse_segments(route, case, i, j, k):
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]

    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        pass
    elif case == OptCase.opt_case_2:
        # A'BC
        solution = list(reversed(first_segment)) + second_segment + third_segment
        
    elif case == OptCase.opt_case_3:
        # ABC'
        solution = first_segment + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_4:
        # A'BC'
        solution = list(reversed(first_segment)) + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_5:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_6:
        # AB'C
        solution = first_segment + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_7:
        # AB'C'
        solution = first_segment + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_8:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_9:
        # A'BC
        solution = list(reversed(third_segment)) + list(reversed(second_segment)) + first_segment
    elif case == OptCase.opt_case_10:
        # ABC'
        solution = third_segment + list(reversed(second_segment)) + list(reversed(first_segment))
    elif case == OptCase.opt_case_11:
        # A'BC'
        solution = third_segment + list(reversed(second_segment)) + first_segment
    elif case == OptCase.opt_case_12:
        # A'B'C
        solution = third_segment + second_segment + list(reversed(first_segment))
    elif case == OptCase.opt_case_13:
        # AB'C
        solution = list(reversed(third_segment)) + second_segment + list(reversed(first_segment))
    elif case == OptCase.opt_case_14:
        # AB'C'
        solution = list(reversed(third_segment)) + second_segment + first_segment
    elif case == OptCase.opt_case_15:
        # A'B'C
        solution = third_segment + second_segment + first_segment
    elif case == OptCase.opt_case_16:
        # A'B'C
        solution = list(reversed(third_segment)) + list(reversed(second_segment)) + list(reversed(first_segment))
    
    #print("sol",solution,cost(danew,solution))
    return solution

# Old Opts

In [ ]:
#THE REMAINING CODE WAS NOT WRITTEN BY ME AND WAS PULLED FROM VARIOUS SOURCES

def two_opt(cost_matrt,routrt):
    routrhold=[]
    routrhold=np.copy(routrt)
    best = routrt.copy()
    improved = True
    while improved:
        improved = False
        for i in range(1, len(routrt)-2):
            for j in range(i+1, len(routrt)+1):
                if j-i == 1: continue # changes nothing, skip then
                new_route = routrt.copy()
                new_route[i:j] = routrt[j-1:i-1:-1] # this is the 2woptSwap
                if cost(cost_matrt, new_route) < cost(cost_matrt, best):
                    best=new_route
                    improved = True
        routrt = best
        
    #print("best",best,"hold",routrhold)
    if cost(cost_matrt, best) > cost(cost_matrt, routrhold):
        return routrhold, cost(cost_matrt, routrhold)
    else:
        return best, cost(cost_matrt, best)




class OptCase(Enum):
    opt_case_1 = "opt_case_1"
    opt_case_2 = "opt_case_2"
    opt_case_3 = "opt_case_3"
    opt_case_4 = "opt_case_4"
    opt_case_5 = "opt_case_5"
    opt_case_6 = "opt_case_6"
    opt_case_7 = "opt_case_7"
    opt_case_8 = "opt_case_8"
    
    

def tsp_3_opt(graph, route):
    import time
    #The following variable was included by me because this 3-opt can get stuck in an infinite loop
    timeout = time.time() + 2
    lengty=len(graph)
    moves_cost = {OptCase.opt_case_1: 0, OptCase.opt_case_2: 0,
                  OptCase.opt_case_3: 0, OptCase.opt_case_4: 0, OptCase.opt_case_5: 0,
                  OptCase.opt_case_6: 0, OptCase.opt_case_7: 0, OptCase.opt_case_8: 0}
    improved = True
    best_found_route = route
    besty_route = route
    stored=cost(graph,best_found_route)
    timeout_start = time.time()
    while improved:
        improved = False
        for (i, j, k) in possible_segments(len(graph)):
            time27 = time.time()
            if time27 > timeout:
                return besty_route
            if(cost(graph,best_found_route)<stored):
                besty_route=best_found_route
            for opt_case in OptCase:
                moves_cost[opt_case] = get_solution_cost_change(graph, best_found_route, opt_case, i, j, k)
            # we need the minimum value of substraction of old route - new route
            best_return = max(moves_cost, key=moves_cost.get)
            if moves_cost[best_return] > 0:
                best_found_route = reverse_segments(best_found_route, best_return, i, j, k)
                improved = True
                break
    # just to start with the same node -> we will need to cycle the results.
    cycled = cycle(best_found_route)
    skipped = dropwhile(lambda x: x != 0, cycled)
    sliced = islice(skipped, None, len(best_found_route))
    best_found_route = list(sliced)
    return best_found_route


def possible_segments(N):
    """ Generate the combination of segments """
    segments = ((i, j, k) for i in range(N) for j in range(i + 2, N-1) for k in range(j + 2, N - 1 + (i > 0)))
    return segments


def get_solution_cost_change(graph, route, case, i, j, k):
    """ Compare current solution with 7 possible 3-opt moves"""
    A, B, C, D, E, F = route[i - 1], route[i], route[j - 1], route[j], route[k - 1], route[k % len(route)]
    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        return 0
    elif case == OptCase.opt_case_2:
        # second case is the case A'BC
        return graph[A, B] + graph[E, F] - (graph[B, F] + graph[A, E])
    elif case == OptCase.opt_case_3:
        # ABC'
        return graph[C, D] + graph[E, F] - (graph[D, F] + graph[C, E])
    elif case == OptCase.opt_case_4:
        # A'BC'
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[A, D] + graph[B, F] + graph[E, C])
    elif case == OptCase.opt_case_5:
        # A'B'C
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[C, F] + graph[B, D] + graph[E, A])
    elif case == OptCase.opt_case_6:
        # AB'C
        return graph[B, A] + graph[D, C] - (graph[C, A] + graph[B, D])
    elif case == OptCase.opt_case_7:
        # AB'C'
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[B, E] + graph[D, F] + graph[C, A])
    elif case == OptCase.opt_case_8:
        # A'B'C
        return graph[A, B] + graph[C, D] + graph[E, F] - (graph[A, D] + graph[C, F] + graph[B, E])

def reverse_segments(route, case, i, j, k):
    if (i - 1) < (k % len(route)):
        first_segment = route[k% len(route):] + route[:i]
    else:
        first_segment = route[k % len(route):i]
    second_segment = route[i:j]
    third_segment = route[j:k]

    if case == OptCase.opt_case_1:
        # first case is the current solution ABC
        pass
    elif case == OptCase.opt_case_2:
        # A'BC
        solution = list(reversed(first_segment)) + second_segment + third_segment
    elif case == OptCase.opt_case_3:
        # ABC'
        solution = first_segment + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_4:
        # A'BC'
        solution = list(reversed(first_segment)) + second_segment + list(reversed(third_segment))
    elif case == OptCase.opt_case_5:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_6:
        # AB'C
        solution = first_segment + list(reversed(second_segment)) + third_segment
    elif case == OptCase.opt_case_7:
        # AB'C'
        solution = first_segment + list(reversed(second_segment)) + list(reversed(third_segment))
    elif case == OptCase.opt_case_8:
        # A'B'C
        solution = list(reversed(first_segment)) + list(reversed(second_segment)) + list(reversed(third_segment))
    return solution
